In [1]:
from frontend.database import session
from scripts import generate_fc_layer_circuit, benchmark_utils
from scripts.circuit import circuit
from scripts.particles import variables, enc_vec
from scripts.reusable_modules import oneb_adder, nb_adder
import numpy as np

In [2]:
from scripts.functions import reduce_add
inp = enc_vec(name='in', nb=4)
print inp.get_variables()
out = enc_vec(name='out', nb=3)
ra = reduce_add('adder', inp, out)
ra_obj = circuit('reduce_add', ra)
ra_obj.write_file(filename='./test.sheep')

['in0745E', 'in1E6A1', 'in2C39A', 'in361A1']


In [3]:
print(inp.get_variables())
print(ra)
print(out.get_variables())

['in0745E', 'in1E6A1', 'in2C39A', 'in361A1']
in0745E in1E6A1 ADD adder_red_out_1_adder_02b_adder_0_tmp11
adder_red_out_1_adder_02b_adder_0_tmp11 ALIAS adder_red_out_10C601866A88
in0745E in1E6A1 MULTIPLY adder_red_out_1_adder_02b_adder_0_tmp26
adder_red_out_1_adder_02b_adder_0_tmp26 ALIAS adder_red_out_1_adder_0_carry_1EE
adder_red_out_1_adder_0_carry_1EE ALIAS adder_red_out_11CE082FE710
in2C39A in361A1 ADD adder_red_out_1_adder_12b_adder_0_tmp1D
adder_red_out_1_adder_12b_adder_0_tmp1D ALIAS adder_red_out_12A91A892E12
in2C39A in361A1 MULTIPLY adder_red_out_1_adder_12b_adder_0_tmp2F
adder_red_out_1_adder_12b_adder_0_tmp2F ALIAS adder_red_out_1_adder_1_carry_196
adder_red_out_1_adder_1_carry_196 ALIAS adder_red_out_13F703CDC920
adder_red_out_10C601866A88 adder_red_out_12A91A892E12 ADD adder_red_out_2_adder_02b_adder_0_tmp12
adder_red_out_2_adder_02b_adder_0_tmp12 ALIAS adder_red_out_202E6932CCC7
adder_red_out_10C601866A88 adder_red_out_12A91A892E12 MULTIPLY adder_red_out_2_adder_02b_adder

In [10]:
import math
nb = 3
depth = int(math.floor(math.log(nb, 2)))
samples=20
inp = enc_vec(name='in', nb=nb)
out = enc_vec(name='out', nb=depth+1)
ra = reduce_add('adder', inp, out)
ra_obj = circuit('reduce_add', ra)
ra_obj.write_file(filename='./test.sheep')
processing_time = np.zeros(samples)
for idx in range(0, samples):
    inp_arr_val = list(map(lambda x : int(x), np.random.uniform(size=nb)> 0.5))
    inp_dict = inp.get_input_dict(inp_arr_val)
    inputs_file = benchmark_utils.write_inputs_file(inp_dict)
    results = benchmark_utils.run_circuit('./test.sheep',inputs_file,"bool","TFHE")
    processing_time[idx] = results['Processing times (s)']['circuit_evaluation']
    out_vars = out.get_variables()
    err_str = str(''.join(results['Outputs'][var] for var in out_vars)[::-1]) +" -- "+ str(np.asarray(inp_arr_val).sum())
    assert int(''.join(results['Outputs'][var] for var in out_vars)[::-1], 2) == np.asarray(inp_arr_val).sum(), err_str
print(processing_time.mean())

0.152057


In [5]:
s = np.asarray([1,2,3])
isinstance(s, np.matrix)

False

In [6]:
s.size

3

In [ ]:
from scripts.nn_layer import linear_layer
nb = 30
samples = 10
inputs = enc_vec(name='nn_input', nb=nb)
depth = int(math.ceil(math.log(nb*1.0, 2)))
outputs = enc_vec(name='nn_outputs', nb=depth+1)
weight = np.asarray(list(map(lambda x : int(x), np.random.uniform(size=nb)> 0.5)))
layer = linear_layer(name='linear', weight=weight,
                    inputs=inputs, outputs=outputs)
layer_obj = circuit('linear_layer', layer)
layer_obj.write_file(filename='./test.sheep')
processing_time = np.zeros(samples)
for idx in range(0, samples):
    inp_arr_val = list(map(lambda x : int(x), np.random.uniform(size=nb)> 0.5))
    #print(inp_arr_val, inp)
    inp_dict = inputs.get_input_dict(inp_arr_val)
    inputs_file = benchmark_utils.write_inputs_file(inp_dict)
    results = benchmark_utils.run_circuit('./test.sheep',inputs_file,"bool","TFHE")
    processing_time[idx] = results['Processing times (s)']['circuit_evaluation']
    out_vars = outputs.get_variables()
    xor_val = (weight == np.asarray([x for x in inp_arr_val]))
    print(weight, inp_arr_val, xor_val)
    err_str = str(''.join(results['Outputs'][var] for var in out_vars)[::-1]) +" -- "+ str(np.asarray(xor_val).sum())
    assert int(''.join(results['Outputs'][var] for var in out_vars)[::-1], 2) == np.asarray(xor_val).sum(), err_str
print(processing_time.mean())

([1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1], <scripts.particles.enc_vec object at 0x10520e690>)
(array([1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0]), [1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1], array([ True, False,  True,  True,  True, False,  True, False, False,
        True, False,  True,  True,  True, False, False,  True, False,
       False, False,  True,  True, False,  True, False,  True,  True,
        True, False, False]))
([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1], <scripts.particles.enc_vec object at 0x10520e690>)
